In [ ]:
#script by Rob Treichler...basic python script score -> 0.45894

#create working directory
import os#Miscellaneous operating system interfaces
os.chdir(r'C:\\Users\\rtreichl\\Documents\\competitions\\BNP Paribas Cardif Claims Management')  #working directory

#%matplotlib inline allows graphics to show below each cell (or graphics in line)
# for some reason, %matplotlib inline won't work if comments are made in the same cell

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd #munging and wrangling
import numpy as np  #for arrays, etc.
import matplotlib.pyplot as plt #graphs/plots

#preprocessing
from sklearn import preprocessing

#Model Selection -> Metrics
from sklearn.metrics import log_loss
from sklearn import cross_validation

#xgboost
import xgboost as xgb #xgboost


In [ ]:
#use dataframes for EDA
train = pd.read_csv('train.csv')
test  = pd.read_csv('test.csv')


In [ ]:
#  Encoding categorical features

for f in train.columns:
    if train[f].dtype=='object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(np.unique(list(train[f].values) + list(test[f].values)))
        train[f] = lbl.transform(list(train[f].values))
        test[f] = lbl.transform(list(test[f].values))

In [ ]:
#flag missing values....these may have predictive value
clist=list(train.columns[2:133])
for i in clist:
    train[i + "_miss"]=np.where(train[i].notnull(),0,1)
    test[i + "_miss"]=np.where(test[i].notnull(),0,1)

#drop variables that don't have missing values
drop_miss_list=[
'v3_miss',
'v22_miss',
'v24_miss',
'v30_miss',
'v31_miss',
'v38_miss',
'v47_miss',
'v52_miss',
'v56_miss',
'v62_miss',
'v66_miss',
'v71_miss',
'v72_miss',
'v74_miss',
'v75_miss',
'v79_miss',
'v91_miss',
'v107_miss',
'v110_miss',
'v112_miss',
'v113_miss',
'v125_miss',
'v129_miss',]

for dml in drop_miss_list:
    try:
        train=train.drop(dml,axis=1)
    except:
        pass
for dml in drop_miss_list:
    try:
        test=test.drop(dml,axis=1)
    except:
        pass

In [ ]:
#impute missing values
#imputation strategy is median

imp = preprocessing.Imputer(strategy='median',axis=1)


for i in train.columns[2:133]:
    temp=pd.DataFrame((imp.fit(train[i]).transform(train[i])).reshape(-1,1))
    train[i]=temp
for i in test.columns[1:132]:
    temp=pd.DataFrame((imp.fit(test[i]).transform(test[i])).reshape(-1,1))
    test[i]=temp
    

In [ ]:
#transform variables to roughly the same scale

#use log base 10 and natural log to tighten things up a bit (outlier management)

#transform log 10
log10_list=['v22','v56','v125','v113']
for l10 in log10_list:
    train[l10]=np.where(train[l10]<= 0,np.log10(np.absolute(train[l10])+1)*-1,np.log10(train[l10]+1))
    test[l10]=np.where(test[l10]<= 0,np.log10(np.absolute(test[l10])+1)*-1,np.log10(test[l10]+1))

#transform ln
ln_list=['v1',
'v2',
'v4',
'v5',
'v6',
'v7',
'v8',
'v9',
'v10',
'v11',
'v12',
'v13',
'v14',
'v15',
'v16',
'v17',
'v18',
'v19',
'v20',
'v21',
'v23',
'v25',
'v26',
'v27',
'v28',
'v29',
'v30',
'v32',
'v33',
'v34',
'v35',
'v36',
'v37',
'v38',
'v39',
'v40',
'v41',
'v42',
'v43',
'v44',
'v45',
'v46',
'v47',
'v48',
'v49',
'v50',
'v51',
'v52',
'v53',
'v54',
'v55',
'v57',
'v58',
'v59',
'v60',
'v61',
'v62',
'v63',
'v64',
'v65',
'v67',
'v68',
'v69',
'v70',
'v71',
'v72',
'v73',
'v76',
'v77',
'v78',
'v79',
'v80',
'v81',
'v82',
'v83',
'v84',
'v85',
'v86',
'v87',
'v88',
'v89',
'v90',
'v91',
'v92',
'v93',
'v94',
'v95',
'v96',
'v97',
'v98',
'v99',
'v100',
'v101',
'v102',
'v103',
'v104',
'v105',
'v106',
'v107',
'v108',
'v109',
'v111',
'v112',
'v114',
'v115',
'v116',
'v117',
'v118',
'v119',
'v120',
'v121',
'v122',
'v123',
'v124',
'v126',
'v127',
'v128',
'v129',
'v130',
'v131',]

for ln in ln_list:
    train[ln]=np.where(train[ln]<= 0,np.log(np.absolute(train[ln])+1)*-1,np.log(train[ln]+1))
    test[ln]=np.where(test[ln]<= 0,np.log(np.absolute(test[ln])+1)*-1,np.log(test[ln]+1))


In [ ]:
#create train and test sets

x_train=train.ix[:,2:241]

y_train=train.ix[:,1:2]

x_test=test.ix[:,1:240]

In [ ]:
#training and test sets for cross validation
x_traincv, x_testcv, y_traincv, y_testcv = cross_validation.train_test_split(x_train,y_train, test_size=0.34, random_state=0)

In [ ]:
#final model with submission file
#def xg_sub(label,x_train,y_train,x_test,parameters):
    #xgb parameters
    
params = {'objective':'binary:logistic','eval_metric' : 'logloss', 'eta': 0.01,
         'subsample':1.0, 'max_depth':9, 'colsample_bytree':0.6,
          'nthread':-1}


    #train input matrix
xgb_train_data = xgb.DMatrix(x_train, y_train)
xgb_test_data  = xgb.DMatrix(x_test)

xgb_estimator = xgb.train(params, xgb_train_data, num_boost_round= 2000)


train_pred=pd.DataFrame(xgb_estimator.predict(xgb_train_data))
test_pred =pd.DataFrame(xgb_estimator.predict(xgb_test_data))

submission_xg = pd.DataFrame(columns=['ID','PredictedProb'])
submission_xg['ID']=test['ID']
submission_xg['PredictedProb']=test_pred.ix[:,0]
submission_xg.to_csv('bnp_2000_1_9_06.csv',index=False)